In [152]:
from py2neo import Graph

In [153]:
import pandas as pd

In [63]:
from searchapp.models import *

In [154]:
graph = Graph("bolt://neo4j:7687", auth=("neo4j", "123qweqwe"))

In [155]:
ids = ['MONDO_5515_oral_cavity_cancer', 'CTD_C024746_tobacco_tar']

In [156]:
hop = 5

In [157]:
query = graph.query(f"""
        MATCH (n)
        RETURN n as node
        LIMIT 10;
    """)

In [158]:
df = query.to_data_frame()

In [159]:
df

,node
0,"{'original_name': 'PHYHIP', 'name': 'phyhip', ..."
1,"{'original_name': 'GPANK1', 'name': 'gpank1', ..."
2,"{'original_name': 'ZRSR2', 'name': 'zrsr2', 'i..."
3,"{'original_name': 'NRF1', 'name': 'nrf1', 'id'..."
4,"{'original_name': 'PI4KA', 'name': 'pi4ka', 'i..."
5,"{'original_name': 'SLC15A1', 'name': 'slc15a1'..."
6,"{'original_name': 'EIF3I', 'name': 'eif3i', 'i..."
7,"{'original_name': 'FAXDC2', 'name': 'faxdc2', ..."
8,"{'original_name': 'MT1A', 'name': 'mt1a', 'id'..."
9,"{'original_name': 'SORT1', 'name': 'sort1', 'i..."


In [160]:
    query = graph.query(
        f"""
            MATCH path = allshortestpaths( (n:Disease)-[rels*..{hop}]->(m:Exposure) )
            WHERE n.id = '{ids[0]}' and m.id = '{ids[1]}'
            RETURN n, rels, m
        """
    )

In [161]:
df = query.to_data_frame()

In [162]:
df

,n,rels,m
0,"{'original_name': 'oral cavity cancer', 'name'...","[{'name': 'parent-child'}, {'name': 'associate...","{'original_name': 'tobacco tar', 'name': 'toba..."


In [228]:
query = graph.query(f"""
        MATCH (n) where n.id IN ["{'","'.join(ids)}"]
        WITH collect(n) as nodes
        UNWIND nodes as n
        UNWIND nodes as m
        WITH * WHERE id(n) > id(m)
        MATCH path = (n)-[r *..{hop}]->(m)
        RETURN path
        ORDER BY n.id DESC
        
    """)

KeyboardInterrupt: 

In [222]:
data = query.data()
data

[{'path': Path(Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer', source='MONDO'), Disease_Disease(Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', original_name='head and neck cancer', source='MONDO'), Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer', source='MONDO'), name='parent-child'), Disease_Protein(Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', original_name='head and neck cancer', source='MONDO'), Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), name='associated with'), Disease_Protein(Node('Disease', id='MONDO_grouped_3781_5607_18597_bronchitis', name='bronchitis', original_name='bronchitis', source='MONDO_grouped'), Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), name='associated with')

In [179]:
for path in [i.get('path') for i in data]:
#     https://py2neo.org/2021.1/data/index.html#py2neo.data.Path
    print(type(path))
    for rel in path.relationships:
        s = dict(rel.start_node)
        p = dict(rel)
        o = dict(rel.end_node)
#         print(s,p,o)
        print(f"{s.get('name')} -- {p.get('name')} --> {o.get('name')}")


<class 'py2neo.data.Path'>
oral cavity cancer -- parent-child --> head and neck cancer
vegfa -- associated with --> head and neck cancer
vegfa -- associated with --> bronchitis
bronchitis -- linked to --> tobacco tar


In [168]:
id = 'MONDO_5515_oral_cavity_cancer'

In [253]:
query = graph.query(f"""
WITH [
    'tobacco tar', 
    'tobacco smoke pollution',
    "Exacerbated by tobacco use",

    'ethanol',

    "anogenital human papillomavirus infection",
    'human papilloma virus infection',
    'human papillomavirus-related squamous cell carcinoma',
    'Persistent human papillomavirus infection',

    'human herpesvirus 8-related tumor',
    
    'herpes simplex virus gingivostomatitis',
    'herpes simplex virus keratitis',

    'syphilis',

    'candidiasis',
    'candida glabrata',
    'candidal paronychia',
    'Recurrent candida infections',
    'oral candidiasis',
    'chronic oral candidiasis',

    'periodontitis',

    'mucocutaneous ulceration, chronic',
    "chronic gingivitis",
    "chronic pain",
    "necrotizing ulcerative gingivitis",

    'bacterial infectious disease with sepsis',

    "vitamin deficiency disorder",

    "arsenic",
    "arsenic acid",
    "arecoline",

    'Formaldehyde',
    'Increased cellular sensitivity to UV light',

    'Sulfur Dioxide',
    'Asbestos',
    'Pesticides',
    'Transport of inorganic cations/anions and amino acids/oligopeptides',
    "methane",
    "hydrocarbons",
    "benzene",
    "coal",
    "gasoline",

    'malignant tumor of floor of mouth',
    'malignant tumor of minor salivary gland',
    'precancerous condition',

    'TP53',
    'GSTM1',
    'EGFR',
    'PRADC1',

    'CD34',
    'ITGA3',
    'ITGBA',
    'VEGFA',
    'GIT1',
    
    "Soil Pollutants, Radioactive ",
    "Radioactive Pollutants ",
    "Radioactive Fallout ",
    "Air Pollutants, Radioactive"
] AS RISK_FACTORS_LIST
UNWIND 
    RISK_FACTORS_LIST AS RISK_FACTORS_UNWIND
WITH 
    toLower(trim(RISK_FACTORS_UNWIND)) AS RISK_FACTORS_LOWERCASE
WITH 
    collect(RISK_FACTORS_LOWERCASE) AS RISK_FACTORS_LOWERCASE_LIST
MATCH 
    path = SHORTESTPATH((risk_fact)-[r*..10]->(n:Disease))
    WHERE 
        n.name = 'oral cavity cancer' and
        toLower(risk_fact.name) IN RISK_FACTORS_LOWERCASE_LIST
RETURN 
    path
    """)

In [254]:
data = query.data()

In [255]:
data

[{'path': Path(Node('Gene/Protein', id='NCBI_1956_EGFR', name='egfr', original_name='EGFR', source='NCBI'), Disease_Protein(Node('Gene/Protein', id='NCBI_1956_EGFR', name='egfr', original_name='EGFR', source='NCBI'), Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', original_name='head and neck cancer', source='MONDO'), name='associated with'), Disease_Disease(Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', original_name='head and neck cancer', source='MONDO'), Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer', source='MONDO'), name='parent-child'))},
 {'path': Path(Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), Disease_Protein(Node('Gene/Protein', id='NCBI_7422_VEGFA', name='vegfa', original_name='VEGFA', source='NCBI'), Node('Disease', id='MONDO_5627_head_and_neck_cancer', name='head and neck cancer', orig

In [283]:
nodes = []
edges = []

In [284]:
for path in [i.get('path') for i in data]:
#     https://py2neo.org/2021.1/data/index.html#py2neo.data.Path
#     print(type(path))
    for rel in path.relationships:
        s = {**rel.start_node, "cluster": str(rel.start_node.labels)}
        p = dict(rel)
        o = {**rel.end_node, "cluster": str(rel.end_node.labels)}
        nodes.append(s)
        nodes.append(o)
        edges.append({
            "source": s.get("id"),
            "target": o.get("id"),
            "label": p.get('name')
        })
#         print(str(rel.start_node.labels))
#         print(f"{s.get('name')} -- {p.get('name')} --> {o.get('name')}")


In [285]:
len(nodes)

426

In [286]:
nodes_unique = [dict(t) for t in {tuple(d.items()) for d in nodes}]
nodes_unique = [{**i, "label": i.get("name")} for i in nodes_unique]
print(len(nodes_unique))
nodes_unique

134


[{'original_name': 'digestive system cancer',
  'name': 'digestive system cancer',
  'id': 'MONDO_2516_digestive_system_cancer',
  'source': 'MONDO',
  'cluster': ':Disease',
  'label': 'digestive system cancer'},
 {'original_name': 'Sulfur Dioxide',
  'name': 'sulfur dioxide',
  'id': 'CTD_D013458_Sulfur_Dioxide',
  'source': 'CTD',
  'cluster': ':Exposure',
  'label': 'sulfur dioxide'},
 {'original_name': 'gingival neoplasm',
  'name': 'gingival neoplasm',
  'id': 'MONDO_21086_gingival_neoplasm',
  'source': 'MONDO',
  'cluster': ':Disease',
  'label': 'gingival neoplasm'},
 {'original_name': 'oral cavity neoplasm',
  'name': 'oral cavity neoplasm',
  'id': 'MONDO_21245_oral_cavity_neoplasm',
  'source': 'MONDO',
  'cluster': ':Disease',
  'label': 'oral cavity neoplasm'},
 {'original_name': 'syphilis',
  'name': 'syphilis',
  'id': 'MONDO_grouped_4497_6718_5976_2897_2896_syphilis',
  'source': 'MONDO_grouped',
  'cluster': ':Disease',
  'label': 'syphilis'},
 {'original_name': 'Arse

In [287]:
edges

[{'source': 'NCBI_1956_EGFR',
  'target': 'MONDO_5627_head_and_neck_cancer',
  'label': 'associated with'},
 {'source': 'MONDO_5627_head_and_neck_cancer',
  'target': 'MONDO_5515_oral_cavity_cancer',
  'label': 'parent-child'},
 {'source': 'NCBI_7422_VEGFA',
  'target': 'MONDO_5627_head_and_neck_cancer',
  'label': 'associated with'},
 {'source': 'MONDO_5627_head_and_neck_cancer',
  'target': 'MONDO_5515_oral_cavity_cancer',
  'label': 'parent-child'},
 {'source': 'NCBI_7157_TP53',
  'target': 'MONDO_5627_head_and_neck_cancer',
  'label': 'associated with'},
 {'source': 'MONDO_5627_head_and_neck_cancer',
  'target': 'MONDO_5515_oral_cavity_cancer',
  'label': 'parent-child'},
 {'source': 'MONDO_5886_oral_candidiasis',
  'target': 'MONDO_6858_mouth_disease',
  'label': 'parent-child'},
 {'source': 'MONDO_6858_mouth_disease',
  'target': 'MONDO_21245_oral_cavity_neoplasm',
  'label': 'parent-child'},
 {'source': 'MONDO_21245_oral_cavity_neoplasm',
  'target': 'MONDO_5515_oral_cavity_canc

In [261]:
labels_qs = Label.objects.all()

In [12]:
for label in labels_qs:
        print(label.name)

Gene/Protein
Drug
Effect/Phenotype
Disease
Biological_Process
Molecular_Function
Cellular_Component
Exposure
Pathway
Anatomy


In [56]:
def _extract_value(elm):
    print(elm)
    return {
        k: v.get('id', v.get('name'))
        for k,v in elm.items()
    }

In [58]:
query = graph.query(f"""
        match r=(head)-[relation]->(tail)
        where head.id = "{ids[0]}" or tail.id = "{ids[0]}"
        return head,relation,tail 
        order by head.id
        limit 20
        
    """)


In [59]:
elm = [_extract_value(elm) for elm in query]

Node('Disease', id='MONDO_21245_oral_cavity_neoplasm', name='oral cavity neoplasm', original_name='oral cavity neoplasm', source='MONDO')	Disease_Disease(Node('Disease', id='MONDO_21245_oral_cavity_neoplasm', name='oral cavity neoplasm', original_name='oral cavity neoplasm', source='MONDO'), Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer', source='MONDO'), name='parent-child')	Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer', source='MONDO')
Node('Disease', id='MONDO_2516_digestive_system_cancer', name='digestive system cancer', original_name='digestive system cancer', source='MONDO')	Disease_Disease(Node('Disease', id='MONDO_2516_digestive_system_cancer', name='digestive system cancer', original_name='digestive system cancer', source='MONDO'), Node('Disease', id='MONDO_5515_oral_cavity_cancer', name='oral cavity cancer', original_name='oral cavity cancer',

In [60]:
elm

[{'head': 'MONDO_21245_oral_cavity_neoplasm',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_2516_digestive_system_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4611_soft_palate_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4631_tongue_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4645_cheek_mucosa_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4669_salivary_gland_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4682_retromolar_area_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4719_hard_palate_cancer',
  'relation': 'parent-child',
  'tail': 'MONDO_5515_oral_cavity_cancer'},
 {'head': 'MONDO_4727_vestibule_of_mouth_cancer',
  '